In [ ]:
import pandas as pd

In [ ]:
import numpy as np

In [ ]:
data = pd.read_csv(r"C:\Users\ps034\Downloads\evride_dataset_5000_records_2025-08-19.csv")

In [ ]:
import os 
print(os.getcwd())

C:\Users\ps034


In [ ]:
print(data.head())

    trip_id       city  distance_km  duration_minutes  traffic_level  \
0  EV000001       Pune        15.23              40.5             60   
1  EV000002     Mumbai        23.72              70.5             60   
2  EV000003      Delhi        30.65             145.8             81   
3  EV000004  Bangalore        51.44             109.9             88   
4  EV000005  Bangalore        16.72              67.7             60   

   demand_factor  battery_health_percent  energy_consumption_kwh  \
0        2.58570                      79                    4.84   
1        1.24488                      82                    5.59   
2        1.98000                      96                    8.57   
3        0.97200                      95                    7.44   
4        2.09300                      90                    6.54   

   route_difficulty vehicle_type  ... weather_condition temperature_celsius  \
0                 4        Sedan  ...             Foggy                  17   


In [ ]:
print(data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 22 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   trip_id                    5000 non-null   object 
 1   city                       5000 non-null   object 
 2   distance_km                5000 non-null   float64
 3   duration_minutes           5000 non-null   float64
 4   traffic_level              5000 non-null   int64  
 5   demand_factor              5000 non-null   float64
 6   battery_health_percent     5000 non-null   int64  
 7   energy_consumption_kwh     5000 non-null   float64
 8   route_difficulty           5000 non-null   int64  
 9   vehicle_type               5000 non-null   object 
 10  time_of_day                5000 non-null   object 
 11  day_of_week                5000 non-null   object 
 12  weather_condition          5000 non-null   object 
 13  temperature_celsius        5000 non-null   int64

In [ ]:
x = data.drop("traffic_level",axis=1)
y= data["traffic_level"]

In [ ]:
print(data.columns)

Index(['trip_id', 'city', 'distance_km', 'duration_minutes', 'traffic_level',
       'demand_factor', 'battery_health_percent', 'energy_consumption_kwh',
       'route_difficulty', 'vehicle_type', 'time_of_day', 'day_of_week',
       'weather_condition', 'temperature_celsius', 'humidity_percent',
       'driver_rating', 'surge_multiplier', 'historical_pricing_factor',
       'is_holiday', 'charging_stations_nearby', 'user_type',
       'fare_amount_inr'],
      dtype='object')


In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
from sklearn.compose import ColumnTransformer

In [ ]:
from sklearn.preprocessing import OneHotEncoder

In [ ]:
from sklearn.impute import SimpleImputer

In [ ]:
from sklearn.pipeline import Pipeline

In [ ]:
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier

In [ ]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score, accuracy_score, f1_score, classification_report
import joblib

In [ ]:
csv_path = (r"C:\Users\ps034\Downloads\evride_dataset_5000_records_2025-08-19.csv")

In [ ]:
data = pd.read_csv(csv_path)

In [ ]:
# optional : quick sanity 

In [ ]:
print("Rows,Cols:",data.shape)
print(data.head(2))

Rows,Cols: (5000, 22)
    trip_id    city  distance_km  duration_minutes  traffic_level  \
0  EV000001    Pune        15.23              40.5             60   
1  EV000002  Mumbai        23.72              70.5             60   

   demand_factor  battery_health_percent  energy_consumption_kwh  \
0        2.58570                      79                    4.84   
1        1.24488                      82                    5.59   

   route_difficulty vehicle_type  ... weather_condition temperature_celsius  \
0                 4        Sedan  ...             Foggy                  17   
1                 4          SUV  ...             Storm                  26   

  humidity_percent  driver_rating  surge_multiplier  \
0               45            4.5              2.47   
1               60            3.6              2.00   

   historical_pricing_factor  is_holiday  charging_stations_nearby  user_type  \
0                       0.94           0                         4    Premium   

In [ ]:
# Basic cleaning


In [ ]:
if 'trip_id' in data.columns:
    data = data.drop(columns=['trip_id'])

In [ ]:
# # Ensure target present

In [ ]:
assert 'fare_amount_inr' in data.columns, "Target column 'fare_amount_inr' missing!"

In [ ]:

# ============ 4) Feature/Target split (for Regression) ============

In [ ]:
y_reg = data['fare_amount_inr']
X= data.drop(columns=['fare_amount_inr'])

In [ ]:
# Categorical vs Numeric columns

In [ ]:
cat_cols = [
    'city', 'traffic_level', 'route_difficulty', 'vehicle_type',
    'time_of_day', 'day_of_week', 'weather_condition', 'user_type'
]

In [ ]:
# Boolean ko numeric banado (0/1). Agar string 'Yes/No' ho to map karlo.

In [ ]:
if 'is_holiday' in X.columns:
    if X['is_holiday'].dtype == bool:
        X['is_holiday'] = X['is_holiday'].astype(int)
    elif X['is_holiday'].dtype == object:
        X['is_holiday'] = X['is_holiday'].map({True:1, False:0, 'Yes':1, 'No':0, 'Y':1, 'N':0}).fillna(0).astype(int)

In [ ]:
# cat_cols me sirf wo rakho jo X me exist karte hain

In [ ]:
cat_cols = [c for c in cat_cols if c in X.columns]

In [ ]:
num_cols = [c for c in X.columns if c not in cat_cols]

In [ ]:
# ============ 5) Preprocessing ==========

In [ ]:
numeric_tf = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="median"))
])

categorical_tf = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="most_frequent")),
    ("onehot", OneHotEncoder(handle_unknown="ignore"))
])

preprocess = ColumnTransformer(
    transformers=[
        ("num", numeric_tf, num_cols),
        ("cat", categorical_tf, cat_cols),
    ],
    remainder="drop"
)

In [ ]:
# ============ 6) Train/Test split ============

In [ ]:
X_train, X_test, y_train_reg, y_test_reg = train_test_split(
    X, y_reg, test_size=0.2, random_state=42
)

In [ ]:
# ============ 7) Random Forest REGRESSION ============

In [ ]:
reg_model = Pipeline(steps=[
    ("prep", preprocess),
    ("rf", RandomForestRegressor(
        n_estimators=300,
        max_depth=None,
        min_samples_split=4,
        min_samples_leaf=1,
        random_state=42,
        n_jobs=-1
    ))
])

In [ ]:
reg_model.fit(X_train, y_train_reg)
y_pred_reg = reg_model.predict(X_test)

In [ ]:
mae = mean_absolute_error(y_test_reg, y_pred_reg)
rmse = mean_squared_error(y_test_reg, y_pred_reg, squared=False)
r2 = r2_score(y_test_reg, y_pred_reg)

C:\Users\ps034\anaconda3\Lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


In [ ]:
print("\n=== REGRESSION METRICS (Fare Prediction) ===")
print(f"MAE : {mae:,.2f}")
print(f"RMSE: {rmse:,.2f}")
print(f"R²  : {r2:,.4f}")


=== REGRESSION METRICS (Fare Prediction) ===
MAE : 432.67
RMSE: 810.93
R²  : 0.8797


In [ ]:
# ============ 8) Feature Importance (top 15) ============
# Get onehot feature names to align with RF importances

In [ ]:
ohe = reg_model.named_steps["prep"].named_transformers_["cat"].named_steps["onehot"]
ohe_features = list(ohe.get_feature_names_out(cat_cols)) if len(cat_cols) else []
all_features = num_cols + ohe_features

In [ ]:
rf = reg_model.named_steps["rf"]
importances = rf.feature_importances_

In [ ]:
fi = pd.DataFrame({
    "feature": all_features,
    "importance": importances
}).sort_values("importance", ascending=False)

In [ ]:
print("\n=== TOP 15 IMPORTANT FEATURES (Regression) ===")
print(fi.head(15).to_string(index=False))


=== TOP 15 IMPORTANT FEATURES (Regression) ===
                  feature  importance
            demand_factor    0.416483
              distance_km    0.344930
     vehicle_type_Premium    0.062146
         surge_multiplier    0.045498
            driver_rating    0.014970
   battery_health_percent    0.012805
   energy_consumption_kwh    0.010272
  weather_condition_Storm    0.008581
historical_pricing_factor    0.008361
         duration_minutes    0.007690
         humidity_percent    0.006947
         vehicle_type_SUV    0.006713
     vehicle_type_Compact    0.006514
      temperature_celsius    0.005933
              city_Mumbai    0.004844


In [ ]:
# ============ 9) Save regression model ============

In [ ]:
joblib.dump(reg_model, "evride_rf_regression.pkl")
print("\nSaved: evride_rf_regression.pkl")


Saved: evride_rf_regression.pkl


In [ ]:
# ============ 10) CLASSIFICATION for Accuracy/F1 (price bands) ============

In [ ]:
# Price ko bands me convert: train set ke quantiles par based to avoid leak

In [ ]:
q = y_train_reg.quantile([0.25, 0.5, 0.75]).values
bins = [-np.inf, q[0], q[1], q[2], np.inf]
labels = ["cheap", "mid", "high", "very_high"]

In [ ]:
# y_class banaye for train/test

In [ ]:
y_train_cls = pd.cut(y_train_reg, bins=bins, labels=labels, include_lowest=True)
y_test_cls  = pd.cut(y_test_reg,  bins=bins, labels=labels, include_lowest=True)

In [ ]:
cls_model = Pipeline(steps=[
    ("prep", preprocess),
    ("rf", RandomForestClassifier(
        n_estimators=400,
        max_depth=None,
        class_weight="balanced",
        random_state=42,
        n_jobs=-1
    ))
])

In [ ]:
cls_model.fit(X_train, y_train_cls)
y_pred_cls = cls_model.predict(X_test)

In [ ]:
acc = accuracy_score(y_test_cls, y_pred_cls)
f1_w = f1_score(y_test_cls, y_pred_cls, average="weighted")

In [ ]:
print("\n=== CLASSIFICATION METRICS (Price Bands) ===")
print(f"Accuracy : {acc:.4f}")
print(f"F1-score : {f1_w:.4f}")
print("\nDetailed report:")
print(classification_report(y_test_cls, y_pred_cls, digits=4))


=== CLASSIFICATION METRICS (Price Bands) ===
Accuracy : 0.7960
F1-score : 0.7936

Detailed report:
              precision    recall  f1-score   support

       cheap     0.8667    0.8735    0.8701       253
        high     0.7635    0.6327    0.6920       245
         mid     0.7044    0.7598    0.7311       254
   very_high     0.8470    0.9153    0.8798       248

    accuracy                         0.7960      1000
   macro avg     0.7954    0.7953    0.7932      1000
weighted avg     0.7953    0.7960    0.7936      1000



In [ ]:
# Save classifier too

In [ ]:
joblib.dump(cls_model, "evride_rf_pricebands_classifier.pkl")
print("\nSaved: evride_rf_pricebands_classifier.pkl")


Saved: evride_rf_pricebands_classifier.pkl
